imports

In [ ]:
# imports
import pandas as pd
import numpy as np
import json

loading data

In [ ]:
# location dataframe
location_df = pd.read_csv('SaO_Optilandia_resub_locations.csv')

# links dataframe
links_df = pd.read_csv('SaO_Optilandia_resub_links.csv')

# extract lorry data from json
lorry_data = json.load(open('SaO_Optilandia_resub_depot_lorries.json', 'r'))

# set count to 0
k = 0

# initialise lorry list
lorry = []

# loop -> set i to the respective lorry key
for i in lorry_data.keys():
    # set j to the the number of lorries at key 'i'
    for j in range(0, len(lorry_data[i])):
        # append each lorry in lorry_data to lorry list
        lorry.append(pd.DataFrame(lorry_data[i][j], index=[k]))
        # accumulate index
        k += 1

# lorry dataframe
lorry_df = pd.concat(lorry)

In [ ]:
# show first 5 rows of lorry_df
lorry_df.head()

In [ ]:
# list of depot locations (where nodes == depot)
depot_locations = np.where(location_df.is_depot)[0]

# list of customer locations (where nodes == customers)
customer_locations = np.where(location_df.is_customer)[0]

imports + visualising the map

In [ ]:
# imports
import networkx as nx
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt 

# parwise distance calculation for each node
euclidean = squareform(pdist(location_df[['x', 'y']]))

# edges list initialisation
edges = []

# loop through links_df rows
for _, (i, j) in links_df.iterrows():
    # append node at i, node at j, and their pairwise distance to edges
    edges.append((i, j, euclidean[i, j]))

# pos dict intialisation
pos = {}

# loop through location_df rows
for k, v in location_df[['x', 'y']].iterrows():
    # update pos dict with array of k, v 
    pos.update({k:v.values})

# initialise depot_labels dict
depot_labels = {}

# loop throgugh depot_locations
for i in depot_locations:
    # update depot_labels dict with {i:i}
    depot_labels.update({i:i})

# initialise customer_labels dict
customer_labels = {}

# loop through customer_locations
for i in customer_locations:
    # update customer_labels dict with {i:i}
    customer_labels.update({i:i})

# initialise nx Graph
G = nx.Graph()

# feed node list to G
G.add_nodes_from(location_df['id'].to_numpy())

# feed edges list to G
G.add_weighted_edges_from(edges)

# resize figure 
plt.figure(figsize=(16, 8))

# sketch graph
nx.draw(G, pos=pos, node_size=40)

# label depot nodes
nx.draw_networkx_labels(G, pos, depot_labels)

# label customer nodes
nx.draw_networkx_labels(G, pos, customer_labels)

# mark depot nodes
nx.draw_networkx_nodes(G, pos=pos, nodelist=depot_locations, node_color='r', node_size=400, alpha=0.9)

# mark customer nodes
nx.draw_networkx_nodes(G, pos=pos, nodelist=customer_locations, node_color='g', node_size=200, alpha=0.3)

clustering nodes (customer_locations) via nearest neighbour sorting

In [ ]:
# intialise cluster dict
cluster = {124:[], 127:[], 167:[], 523:[]}

# intialise nodes list
nodes = [] 

# loop through each node in customer_locations
for node in customer_locations:
    # check if node in nodes
    if node not in nodes:
        # initialise dist list
        dist = []
        # loop through each depot key
        for depot in cluster.keys():
            # append euclidean weights to dist 
            dist.append(euclidean[node, depot])
        # get shortest distance
        shortestDist = min(dist)
        # match shortest distance to equivalent node index
        nearestDepotIndex = np.where(euclidean[node]==shortestDist)
        # add node to relative nearest depot location
        cluster[int(nearestDepotIndex[0])].append(node)
        # track applied nodes
        nodes.append(node)
        # clear dist
        dist.clear()

# print allocated nodes to relative cluster points (depot locations)
print(cluster)

# clear nodes list
nodes.clear()

states and constraints

In [ ]:
# setting required column 
location_df['required'] = location_df['capacity']-location_df['level']

# displaying rows where is_customer true
location_df[location_df['is_customer']==True].head()

In [ ]:
# function: finding next nearest customer node
def nearest_customer(currentState, customerList):
    #initialise dist dict
    dist = {}
    
    # loop through customerList
    for i in customerList:
        # check for all where customer != currentState
        if i != currentState:
            # update dist with available customer index and their relative weights
            dist.update({i:euclidean[i,currentState]})

    # initialise temp list
    temp = []

    # loop through keys of dist 
    for i in dist.keys():
        # add weights to temp
        temp.append(dist[i])
    
    # get lowest weight
    _shortestDist = min(temp)
    
    # find relative index of lowest weight
    nearestCustomerIndex = np.where(euclidean[currentState]==_shortestDist)
    
    # return next index with relative weight
    return int(nearestCustomerIndex[0]), _shortestDist

In [ ]:
# test nearest_customer()
nearest_customer(1, [2, 3, 1, 5])

In [ ]:
# function: finding the nearest depot
def nearest_depot(currentState, depotList):
    # intialise dist dict
    dist = {}

    # loop through depotList
    for i in depotList:
        # update dist with depot and their relative distance values
        dist.update({i:euclidean[currentState, i]})
    
    # intialise temp list
    temp = []

    # loop through dist.keys()
    for i in dist.keys():
        # add values of each dist.keys() to temp 
        temp.append(dist[i])

    # get lowest depot weight
    _shortestDist = min(temp)

    # get the relative node index of the closest depot
    _nearestDepotIndex = np.where(euclidean[currentState]==_shortestDist)

    # return next depot index with relative weight
    return int(_nearestDepotIndex[0]), _shortestDist

In [ ]:
# test near_depot()
nearest_depot(1, depot_locations)

greedy search (best-first-search)

In [ ]:
# intialise routes dict
routes = {}

# loop through lorry_df.index 
for i in lorry_df.index:
    # update routes with key: lorry_id, value: capacity
    routes.update({lorry_df['lorry_id'][i]:lorry_df['capacity'][i]})

In [ ]:
# display journeys so far
routes

In [ ]:
print(edges[0][2])

In [92]:
def routing(currentState, targetState, edges):
    nodes = [currentState]
    storedEdges = edges 
    currentNode = currentState
    targetNode = targetState
    toggleEdges = list(np.where(links_df[['id1', 'id2']]==currentNode)[0])
    traversedEdges = []
    routeWeights = []

    for edge in toggleEdges:
        while currentNode != targetNode:
            if storedEdges[edge][0] == targetNode or storedEdges[edge][1] == targetNode:
                print(f'targetNode found:')
                traversedEdges.append(edge)
                currentNode = targetNode
                nodes.append(currentNode)
                routeWeights.append(storedEdges[edge][2])
            else:
                print(f'targetNode not found:')
                print(f'currentNode: {currentNode}; searching for path')
                print(f'current edge: {edge}')
                print(f'nodes so far: {nodes}')
                toggleEdges = list(np.where(links_df[['id1', 'id2']]==currentNode)[0])
                print(f'toggleEdges {toggleEdges}')
                edgeWeights = []
                visitedToggleEdge = []
                for i, secEdge in enumerate(toggleEdges):
                    if secEdge in traversedEdges:
                        edgeWeights.append(storedEdges[secEdge][2])
                        visitedToggleEdge.append(secEdge)
                    if secEdge not in traversedEdges:
                        edgeWeights.append(storedEdges[secEdge][2])
                print(f'edgeWeights: {edgeWeights}')
                print(f'visitedToggleEdge: {visitedToggleEdge}')
                
# # # Hitting a *DEADEND* PROBLEM <-- could be as we are using EDGE and not NODE (NOT SURE IF WE ARE) for path update?
                for i, visitedEdge in enumerate(visitedToggleEdge):
                    if visitedEdge in toggleEdges:
                        weightToRemove = storedEdges[visitedEdge][2]
                        toggleEdges.remove(visitedEdge)
                        edgeWeights.remove(weightToRemove)
          
                print(f'toggleEdges post-remove(): {toggleEdges}')
                print(f'edgeWeights post-pop(): {edgeWeights}')

                # index of smallest routeWeights value in toggleEdges
                nextBestIndex = edgeWeights.index(min(edgeWeights))
                # actual edge index of toggleEdges and links_df 
                relativeBestIndex = toggleEdges[nextBestIndex]
                # nextBestEdge = np.where(links_df.index==nextBestIndex)[0][0]
                nextBestEdge = np.where(links_df.index==relativeBestIndex)[0][0]
                print(f'selected Edge = {nextBestEdge}')
                # tempEdgeIndex = toggleEdges[nextBestEdge]
                bestEdgeNodes = storedEdges[nextBestEdge]

                if currentNode == bestEdgeNodes[0] and currentNode != bestEdgeNodes[1]:
                    print(f'id1 selected')
                    currentNode = bestEdgeNodes[1]
                    # edge = tempEdgeIndex 
                    edge = nextBestEdge
                    nodes.append(currentNode)
                    # traversedEdges.append(tempEdgeIndex)
                    traversedEdges.append(nextBestEdge)
                    print(f'traversedEdges post-id1: {traversedEdges}')
                    # routeWeights.append(storedEdges[tempEdgeIndex][2])
                    routeWeights.append(storedEdges[nextBestEdge][2])
                    print(f'routeWeights post id1: {routeWeights}')
                    print(f'\n')
                elif currentNode != bestEdgeNodes[0] and currentNode == bestEdgeNodes[1]:
                    print(f'id2 selected')
                    currentNode = bestEdgeNodes[0]
                    # edge = tempEdgeIndex
                    edge = nextBestEdge
                    nodes.append(currentNode)
                    # traversedEdges.append(tempEdgeIndex)
                    traversedEdges.append(nextBestEdge)
                    print(f'traversedEdges post-id2: {traversedEdges}')
                    # routeWeights.append(storedEdges[tempEdgeIndex][2])
                    routeWeights.append(storedEdges[nextBestEdge][2])
                    print(f'routeWeights post id1: {routeWeights}')
                    print(f'\n')

        break

routing(124, 572, edges)

targetNode not found:
currentNode: 124; searching for path
current edge: 308
nodes so far: [124]
toggleEdges [308, 309, 310]
edgeWeights: [38.66485532791097, 48.89433013699728, 33.06741368345007]
visitedToggleEdge: []
toggleEdges post-remove(): [308, 309, 310]
edgeWeights post-pop(): [38.66485532791097, 48.89433013699728, 33.06741368345007]
selected Edge = 310
id1 selected
traversedEdges post-id1: [310]
routeWeights post id1: [33.06741368345007]


targetNode not found:
currentNode: 542; searching for path
current edge: 310
nodes so far: [124, 542]
toggleEdges [310, 677, 834]
edgeWeights: [33.06741368345007, 38.237537896097784, 32.851606238387085]
visitedToggleEdge: [310]
toggleEdges post-remove(): [677, 834]
edgeWeights post-pop(): [38.237537896097784, 32.851606238387085]
selected Edge = 834
id2 selected
traversedEdges post-id2: [310, 834]
routeWeights post id1: [33.06741368345007, 32.851606238387085]


targetNode not found:
currentNode: 473; searching for path
current edge: 834
nodes 

ValueError: min() arg is an empty sequence

In [ ]:
from dijkstar import Graph, find_path

graph = Graph()

for _, (i, j) in links_df.iterrows():
    graph.add_edge(i, j, _)

find_path(graph, 0, 605)
# find_path(graph, 124, 10)

In [ ]:
temp = [0, 1, 2, 3, 4]

def route(lorryId, allocatedCustomers):
    currentId = lorryId
    operatingDepot = lorryId.split('-')[0]
    currentCapacity = float(lorry_df[lorry_df['lorry_id']==lorryId].capacity)
    currentCpm = float(lorry_df[lorry_df['lorry_id']==lorryId].cpm)
    currentState = int(operatingDepot)
    nextCustomerCapacity = 0
    while currentCapacity >= nextCustomerCapacity and len(allocatedCustomers)>0:
        nextCustomer = nearest_customer(currentState, allocatedCustomers)
        print(nextCustomer)
        # need for a routing function ... 
        print(allocatedCustomers)
        break


route("523-0", temp)